In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import os
import gc
import tqdm
import datetime
import random
from collections import defaultdict
from sklearn.neural_network import MLPRegressor

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit, GroupKFold
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor

# model
import xgboost as xgb
import lightgbm as lgb

# evaluation
from sklearn.metrics import mean_squared_error

# install
!pip install workalendar
from workalendar.asia import SouthKorea

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 163kB 5.1MB/s 
     |████████████████████████████████| 225kB 39.8MB/s 
     |████████████████████████████████| 16.0MB 156kB/s 
  Created wheel for skyfield: filename=skyfield-1.13-cp36-none-any.whl size=253163 sha256=c0045bf653e705f853ab95aace86e094900a8595fb8d64f4748d1681e9689210
  Stored in directory: /root/.cache/pip/wheels/76/3d/1c/afe30b6c7a526ba23b63c1947c64cc7b9142a6e78858fe2384
  Created wheel for pyCalverter: fi

In [0]:
path = 'drive/My Drive/11dacon/data/'

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    
from numba import jit
import math

@jit
def smape_fast(y_true, y_pred, exp=False):
    
    if exp:
        y_true = np.expm1(np.array(y_true))
        y_pred = np.expm1(np.array(y_pred))
    else:
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

def rmse(y_true, y_pred, exp=False):
    if exp:
        return np.sqrt(mean_squared_error(np.expm1(y_true), np.expm1(y_pred)))
    else:
        return np.sqrt(mean_squared_error(y_true, y_pred))

In [0]:
test = pd.read_csv(path+'test.csv')
sub = pd.read_csv(path+'submission.csv')

holidays = pd.concat([pd.Series(np.array(SouthKorea().holidays(2018))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2017))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2016))[:, 0])]).reset_index(drop=True)

In [0]:
weather = pd.read_csv(path+'weather_day.csv', encoding='cp949').iloc[:, 1:]
weather.columns = ['date', '평균기온', '최저기온', '최고기온']
weather['date'] = pd.to_datetime(weather['date'])

In [0]:
def merge(train, col):
    temp = train[['Time', col]].rename(columns={col:'target'})
    temp['Time'] = pd.to_datetime(temp['Time'])
    temp = temp[temp['Time']>='2017-11-23'].reset_index(drop=True)
    
    temp['date'] = pd.to_datetime(temp['Time'].dt.date)
    temp = temp.groupby('date')['target'].sum().reset_index()
    temp['Time'] = pd.to_datetime(temp['date'].dt.date)
    temp = temp[temp['target']>0].reset_index(drop=True)
    temp = temp[(temp['target']>temp['target'].mean() - 3*temp['target'].std()) & (temp['target']<temp['target'].mean() + 3*temp['target'].std())]
    
    
    temp['Time'] = pd.to_datetime(temp['Time'])
    temp['month'] = temp['Time'].dt.month
    temp['week'] = temp['Time'].dt.week
    temp['weekday'] = temp['Time'].dt.weekday
    temp['day'] = temp['Time'].dt.day
    temp['hour'] = temp['Time'].dt.hour
    temp['holiday'] = temp['Time'].dt.date.isin(holidays).astype(int)
    temp['weekend'] = temp['weekday'].map({0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1})
    temp['is_holiday'] = (temp['weekend'] + temp['holiday']).map({0:0, 1:1, 2:1})
    

    
    temp2 = pd.DataFrame(pd.date_range('20180701', '20181201', freq='h'), columns=['Time']).iloc[:-1, :]
    temp2['Time'] = pd.to_datetime(temp2['Time'])
    temp2['month'] = temp2['Time'].dt.month
    temp2['week'] = temp2['Time'].dt.week
    temp2['weekday'] = temp2['Time'].dt.weekday
    temp2['day'] = temp2['Time'].dt.day
    temp2['hour'] = temp2['Time'].dt.hour
    temp2['holiday'] = temp2['Time'].dt.date.isin(holidays).astype(int)
    temp2['weekend'] = temp2['weekday'].map({0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1})
    temp2['is_holiday'] = (temp2['weekend'] + temp2['holiday']).map({0:0, 1:1, 2:1})
    
    temp = pd.merge(temp, weather, how='left', on='date')
    
    return temp, temp2

In [0]:
test_df = train_df.iloc[-10:]
train_df = train_df.iloc[:-10]

In [0]:
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
import itertools

import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

In [0]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))

In [0]:
# setting initial values and some bounds for them
ps = range(2, 5)
d=1 
qs = range(2, 5)
Ps = range(0, 2)
D=1 
Qs = range(0, 2)
s = 24 # season length is still 24

# creating list with all the possible combinations of parameters
parameters = itertools.product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

def optimizeSARIMA(parameters_list, d, D, s):
    """
        Return dataframe with parameters and corresponding AIC
        
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order in ARIMA model
        D - seasonal integration order 
        s - length of season
    """
    
    results = []
    best_aic = float("inf")

    for param in tqdm.tqdm_notebook(parameters_list):
        # we need try-except because on some combinations model fails to converge
        try:
            model=sm.tsa.statespace.SARIMAX(train_df['target'], order=(param[0], d, param[1]), 
                                            seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
        aic = model.aic
        # saving best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])

    result_table = pd.DataFrame(results)
    result_table.columns = ['parameters', 'aic']
    # sorting in ascending order, the lower AIC is - the better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    
    return result_table

In [117]:
result_table = optimizeSARIMA(parameters_list, d, D, s)
# set the parameters that give the lowest AIC
p, q, P, Q = result_table.parameters[0]

best_model=sm.tsa.statespace.SARIMAX(train_df['target'], order=(p, d, q), 
                                        seasonal_order=(P, D, Q, s)).fit(disp=-1)
forecast = best_model.predict(start = train_df.shape[0], end = train_df.shape[0]+10)
forecast

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:949: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:961: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimiza

63    140.325901
64    146.477015
65    146.203582
66    145.113623
67    149.838902
68    144.489140
69    143.666079
70    145.441570
71    148.451993
72    145.544554
73    143.735728
dtype: float64

In [118]:
test_df.target

63    139.486
64    144.542
65    147.105
66    151.940
67    154.992
68    147.142
69    146.831
70    146.159
71    152.207
72    154.966
Name: target, dtype: float64

In [0]:
model = ARIMA(train_df['target'], order=get_optimal_params(train_df['target']))
results_ARIMA = model.fit(disp=-1)
results_ARIMA.forecast(10)

In [112]:
SEED=42
seed_everything(SEED)

params = {
    'objective':'regression',
    'boosting_type':'gbdt',
    'metric':'rmse',
    'n_jobs':-1,
    'learning_rate':0.03,
    'num_leaves': 2**9,
    'max_depth':-1,
    'tree_learner':'serial',
    'min_child_weight':5, 
    'subsample':0.7,
    'reg_alpha':0.1,
    'reg_lambda':0.1,
    'verbose':-1,
    'seed': SEED
}
    
rmse_list = []
smape_list = []
submit_dict = defaultdict()

target_arr = np.array([])
oof_arr = np.array([])

mode = 'test'
if mode=='validation':
    val_oof = np.zeros(len(valid_df))
    val_target = np.array([])
    val_oof_list = np.array([])

    val_rmse_list = []
    val_smape_list = []


for idx in tqdm.tqdm_notebook(list(range(200))):
    train_df, test_df = merge(test, sub['meter_id'][idx])
    
    if mode=='validation':
        valid_df = train_df.iloc[-24:]
        train_df = train_df.iloc[:-24]
    
    oof = np.zeros(len(train_df))
    pred = np.zeros(len(test_df))
    
    feature = [i for i in train_df.columns if i not in ['target', 'Time', 'weekend', 'holiday', 'date']]
    feature = [i for i in feature if 'diff' not in i]
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    if idx==1:
        break

    for trn_idx, val_idx in kf.split(train_df):
        tt = lgb.Dataset(train_df.loc[trn_idx, feature], train_df.loc[trn_idx, ['target']])
        vv = lgb.Dataset(train_df.loc[val_idx, feature], train_df.loc[val_idx, ['target']])

        model = lgb.train(params, tt, valid_sets=[tt, vv], early_stopping_rounds=50, verbose_eval=0)
#         model = SVR(degree=3, coef0=0.001, kernel='rbf', gamma='auto').fit(train_df.loc[trn_idx, feature], train_df.loc[trn_idx, 'target'])
#         model = RandomForestRegressor(n_estimators=100, random_state=42).fit(train_df.loc[trn_idx, feature], train_df.loc[trn_idx, 'target'])

        oof[val_idx] = model.predict(train_df.loc[val_idx, feature])
        pred += model.predict(test_df[feature])/5
    
        if mode=='validation':
            val_oof += model.predict(valid_df[feature])/5
    
    if mode=='validation':
        val_target = np.concatenate([val_target, valid_df['target'].values])
        val_oof_list = np.concatenate([val_oof_list, val_oof])
        print(idx, smape_fast(val_target, val_oof_list), rmse(val_target, val_oof_list))
        
        val_rmse_list.append(rmse(val_target, val_oof_list))
        val_smape_list.append(smape_fast(val_target, val_oof_list))
    
    oof[oof<0] = train_df['target'].min()
    pred[pred<0] = train_df['target'].min()
#     oof[oof<0] = 0
#     pred[pred<0] = 0
    
    target_arr = np.concatenate([target_arr, train_df['target'].values])
    oof_arr = np.concatenate([oof_arr, oof])
    print(idx, smape_fast(target_arr, oof_arr), rmse(target_arr, oof_arr), smape_fast(train_df['target'], oof), rmse(train_df['target'], oof))
    
    # 할당
    rmse_list.append(rmse(train_df['target'], oof))
    smape_list.append(smape_fast(train_df['target'], oof))
    submit_dict[idx] = pred

KeyError: ignored

In [0]:
rf: 199 193.91180678719994 122.26467220592971
lgb : 199 193.8824586822974 118.74868605407516

In [0]:
np.mean(rmse_list), np.mean(smape_list)

(0.2660960687585986, 29.514586692448574)

In [0]:
np.mean(rmse_list), np.mean(smape_list)

(0.2715865953678639, 29.698993530910787)

In [0]:
submit_df = pd.concat([pd.DataFrame(submit_dict).loc[:23], 
                       pd.concat([pd.DataFrame([j for i in range(10) for j in str(i) * 24], columns=['house']), pd.DataFrame(submit_dict).loc[:239]], 1).groupby('house').sum().reset_index(drop=True),
                       pd.concat([test_df['Time'].dt.to_period('m'), pd.DataFrame(submit_dict)], 1).groupby('Time').sum().reset_index(drop=True)])

submit_df.columns = sub['meter_id']
submit_df = submit_df.T.reset_index()
submit_df.columns = sub.columns
submit_df.head(15)

In [0]:
-----------------------------------------------------------------
month, day, week, weekday, hour 만 사용

nan process -> 3sigma
mean : (0.2678278947214462, 29.149798158558028)
LB : 29.8

+ is_holiday => 광한
mean : (0.2562240039588324, 28.834643183490357)
LB : 29.59
    
- is_holiday
mean : (0.2715865953678639, 29.698993530910787)
LB : 29.610441
    
+ is_holiday
LB : 29.463389
    

+ continuous equal value
mean : (0.2642950983875577, 29.08533340630285) : nmax=3, 
        (0.2635035317889631, 29.034958873968534) : nmax=4, 
        (0.263370016717205, 29.05395734892656) : nmax=5,
        (0.26332217195667157, 29.041987147018972) : nmax=6,
        (0.2629562507335404, 29.004030535332717) : nmax=7,
LB : 
    
+ continuous equal value + is_holiday => 은선
mean : (0.2572715264515067, 28.8154113691253) : nmax=7,
        (0.25646055098412307, 28.823119972877404) : nmax=25
LB : 29.753622(nmax=7 version)
    


min
mean : (0.25621980207812756, 28.69560261882844)
LB : 
    
min, working time
mean : (0.2566664670312808, 28.68228620004992)
LB : 


    
    
    

1. 실험요소, nan값 많은, prior target 지우는가?
2. -prediction 0 or min : (0.25621980207812756, 28.69560261882844), 29.11
3. feature combine
4. weather data
5. xgb, seed ensemble



min : np.mean(rmse_list), np.mean(smape_list)  
    
    
nan process(dropna version and prior target) -> 3sigma
mean : (0.2617307645486781, 29.029936608903007)
LB : 
    
    
nan process(dropna version and prior target) -> 3sigma + -prediction.fillna(train_df['target'].min())
mean : (0.2617263166484412, 28.885100489166994)
LB : 

In [0]:
from google.colab import drive
drive.mount('/content/drive')
path = 'drive/My Drive/11dacon/submit/'

submit_df.to_csv(path+'holiday3.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
submit_df.head(15)